In [1]:
import numpy as np
import math
import torch
import random

import matplotlib.pyplot as plt

import tqdm
from tqdm import trange

import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(0)
#permet davoir des resultats reproducibles

In [3]:



class TicTacToe:
    def __init__(self):
        self.row_count = 3
        self.column_count = 3
        self.action_size = self.row_count * self.column_count
        
    def get_initial_state(self):
        return np.zeros((self.row_count, self.column_count))
    
    
    def get_next_state(self, state, action, player):
        row = action // self.column_count
        column = action % self.column_count
        state[row, column] = player
        return state
    #action entier entre 0 et 8
    
    def get_valid_moves(self, state):
        return (state.reshape(-1) == 0).astype(np.uint8)
    #state.reshape(2,3) transforme un tenseur en matrice 2 par 3
    #state.reshape(-1) laisse pytorch decider quelle est la bonne taille
    #renvoie un tableau de booleens selon si on peut jouer dans une case ou non
    # mais cest converti en 0 ou 1 par astype 
    
    def check_win(self, state, action):
        if action == None:
            return False
        
        row = action // self.column_count
        column = action % self.column_count
        player = state[row, column]
        
        return (
            np.sum(state[row, :]) == player * self.column_count
            #state[row, :] renvoie un tab contenant toute la ligne row
            or np.sum(state[:, column]) == player * self.row_count
            or np.sum(np.diag(state)) == player * self.row_count
            or np.sum(np.diag(np.flip(state, axis=0))) == player * self.row_count
        )
    
    
    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        return 0, False
    
    def get_opponent(self, player):
        return -player
    
    def get_opponent_value(self, value):
        return -value
    
    def change_perspective(self, state, player):
        return state * player
    
    def get_encoded_state(self, state):
        encoded_state = np.stack(
            (state == -1, state == 0, state == 1)
        ).astype(np.float32)
        
        return encoded_state
    #ceci transforme le plateau (tab) en trois tab : j1, j2, vide

In [4]:
tab = np.random.rand(3,3)
print (tab)
tab[(2,0)]
any([-1,-1])
() + (0,)

[[0.01448125 0.49115843 0.67838809]
 [0.67212423 0.0539281  0.25638872]
 [0.62807648 0.49253838 0.08341415]]


(0,)

In [5]:
#le but netant pas de jouer mais detablier lexistence de strategie
# gagnante, on peut implementer une version peu intuitive mais plus simple a implementer et donc en 2D
import itertools

                

    
    
class NDGame:
    def __init__(self, length, dim):
        self.length = length # elgnth as in the length of the side of the hypercube
        self.dim = dim
        self.action_size = self.length ** self.dim


    def get_initial_state(self):
        dimensions = (self.length, ) * self.dim #dim-uplet fait de chaque bord de lhypercube
        return np.zeros(dimensions) 
            #jimplemente un matrice taille^n mais je la transformerai en 2D plus tard
            
            #implementation par hypergraphe etait lautre moyen 
            #on aurait fait ca pour garder la pertinence de conv2D
            #la on met la fonction arete de william pour inserer des 2
            
    def to_base(self, n, base):
        decomp = []
        for _ in range(self.dim):
            decomp.append(n % base)
            n //= base
        return tuple(reversed(decomp))

    
    def get_next_state(self, state, action, player):
        indices = self.to_base(action, self.length)
        state[indices] = player
        return state

    def get_valid_moves(self, state):
        return (state.reshape(-1) == 0).astype(np.uint8)
        #transforme mon tableau n dimensionsel en tab 1 D trouve les cases libres (==0) 
        #ca renvoie un tableau de 1 et 0 selon si la case est libre

    def get_opponent(self, player):
        return -player

    def get_opponent_value(self, value):
        return -value

    def change_perspective(self, state, player):
        return state * player

    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        return 0, False

    def get_directions(self):
        return [d for d in itertools.product([-1, 0, 1], repeat=self.dim) if any(d)]
        #any regarde si il ny a que des zero (cest un OU pour chaque element du tableau en quelque sorte)
    
    def in_bounds(self, indices):
        return all(0 <= i < self.length for i in indices)

    def check_win(self, state, action):
        if action is None:
            return False
        
        indices_action = self.to_base(action, self.length)
        player = state[indices_action]
        for direction in self.get_directions():
            indices_avant = indices_action
            indices_arriere = indices_action
            alignes = 1
            for i in range(self.length):
                
                indices_avant = tuple((x + y)  for x, y in zip(indices_avant, direction))
                if self.in_bounds(indices_avant):
                    alignes += (state[indices_avant] == player)
                    
                indices_arriere = tuple((x - y)  for x, y in zip(indices_arriere, direction))
                if self.in_bounds(indices_arriere):
                    alignes += (state[indices_arriere] == player)
                    
            if alignes >= self.length :
                return True
        return False

    def get_encoded_state(self, state):
        state = state.reshape(self.length,-1)
        encoded_state = np.stack(
            (state == -1, state == 0, state == 1)
        ).astype(np.float32)
        
        return encoded_state
    

In [6]:

nd = NDGame(3,3)
etat = nd.get_initial_state()
print(etat)
etat = nd.get_next_state(etat, 0, 1)
etat = nd.get_next_state(etat, 2, 1)
etat = nd.get_next_state(etat, 3, 1)
etat = nd.get_next_state(etat, 7, 1)
etat = nd.get_next_state(etat, 4, -1)
etat = nd.get_next_state(etat, 5, -1)
etat = nd.get_next_state(etat, 6, 1)

print("\n")
print(etat)

arr = nd.get_valid_moves(etat)
indices = np.where(arr == 1)[0]        # prend les indices valant 1
random_index = np.random.choice(indices)
print(random_index)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


[[[ 1.  0.  1.]
  [ 1. -1. -1.]
  [ 1.  1.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]
12


In [7]:
class ResNet(nn.Module):
    def __init__(self, game, num_resBlocks, num_hidden, device):
        #nb de convolutional block ie longueur du backbone
        # et leur taille
        
        super().__init__()
        #super verifie que tout le system est correctement mis en place
        #ie il inititalise nn.module
        
        self.device = device
        
        self.startBlock = nn.Sequential(
            nn.Conv2d(3, num_hidden, kernel_size=3, padding=1),
            #cest une couche convolutionnelle
            
            #input channels
            #le premier 3 permet de prendre trois grilles en parametre : j1, j2, vide
            
            #output channels
            #num hidden est le nb de filtres, de feature maps
            
            # idee : implementer en n dimension ?
            
            #kernel size donne la taille des filtres ie la taille de lendroit zoomé
            nn.BatchNorm2d(num_hidden),
            #normalisation augmente la vitesee dentrainement askip
            nn.ReLU()
            #enleve les valeurs negatives
        )
        #nn.sequential permet de definir une sequence de fonctions a appliquer
        #ca cree donc un nouveau bloc comme suite des trois differents blocs
        
        self.backBone = nn.ModuleList(
            [ResBlock(num_hidden) for i in range(num_resBlocks)]
            #crée num_resBlocks residual blocks, chacun avec num_hidden filtres,
            #et les garde comme liste de modules a entrainer
            #comme partie principale du nn : comme backbone
        )
        
        self.policyHead = nn.Sequential(
            nn.Conv2d(num_hidden, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            # 4D : (batch_size, channels, height, width) -> 1D
            nn.Linear(32 * game.action_size, game.action_size)
            #fully connected layer
        )#32 est arbitraire mais empiriquement pas mauvais
        
        self.valueHead = nn.Sequential(
            nn.Conv2d(num_hidden, 3, kernel_size=3, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3 * game.action_size, 1),
            nn.Tanh()
        )
        
        self.to(device)
        #deplace le modele vers le device : GPU/CPU
        
    def forward(self, x):
        x = self.startBlock(x)
        for resBlock in self.backBone:
            x = resBlock(x)
        policy = self.policyHead(x)
        value = self.valueHead(x)
        return policy, value
    
class ResBlock(nn.Module):
    def __init__(self, num_hidden):
        super().__init__()
        self.conv1 = nn.Conv2d(num_hidden, num_hidden, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_hidden)
        self.conv2 = nn.Conv2d(num_hidden, num_hidden, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_hidden)

    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x
        #on applique le saut : sortie = entree + F(entree) 


In [8]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None, prior = 0, visit_count = 0):
        
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        self.prior = prior
        #prior : proba d'une action du pov du parent
        self.children = []
        self.visit_count = visit_count
        """
        self.expandable_moves = game.get_valid_moves(state)
        #car on expande tout dun coup mtn
        """
        self.visit_count = visit_count
        self.value_sum = 0
        
    def is_fully_expanded(self):
        return len(self.children) > 0 # and np.sum(self.expandable_moves) == 0 
    
    def select(self):
        best_child = None
        best_ucb = -np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
                
        return best_child
    
    def get_ucb(self, child):
        if child.visit_count == 0:
            q_value  = 0
        else:
            q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        # 1 - ... permet de renverser les roles parents/enfant adversaires 
        #return q_value + self.args['C'] * (math.sqrt(math.log(self.visit_count) / child.visit_count)
        return q_value + self.args['C'] * (math.sqrt(self.visit_count) / (child.visit_count + 1)) * child.prior
    
    def expand(self, policy):
        for action, prob in enumerate(policy):
            if prob > 0:
                child_state = self.state.copy()
                child_state = self.game.get_next_state(child_state, action, 1)
                child_state = self.game.change_perspective(child_state, player=-1)

                child = Node(self.game, self.args, child_state, self, action, prob)
                self.children.append(child)
        return child
        """
        action = np.random.choice(np.where(self.expandable_moves == 1)[0])
        # [0] tranforme le n - uplet renvoyé en un tab auquel rand choice est appliquable
        self.expandable_moves[action] = 0
        #indique que la case est desormais plus jouable
        """
        
                    
        
    """
    def simulate(self):
        value, is_terminal = self.game.get_value_and_terminated(self.state, self.action_taken)
        value = self.game.get_opponent_value(value)
        
        if is_terminal:
            return value
        
        rollout_state = self.state.copy()
        rollout_player = 1
        #rollout state signifie etat de reference, courant, initial 
        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = np.random.choice(np.where(valid_moves == 1)[0])
            rollout_state = self.game.get_next_state(rollout_state, action, rollout_player)
            value, is_terminal = self.game.get_value_and_terminated(rollout_state, action)
            if is_terminal:
                if rollout_player == -1:
                    value = self.game.get_opponent_value(value)
                return value    
            
            rollout_player = self.game.get_opponent(rollout_player)
    """
    
    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        
        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)  


class MCTS:
    def __init__(self, game, args, model):
        self.game = game
        self.args = args
        self.model = model
        
    @torch.no_grad() 
    # car on ne souhaite pas stocker le parcours effectué sur le 
    # gradient mais juste lutiliser pour obtenir policy,val
    # enleve lautograd 
    #gain de memoire et de vitesse
    
    def search(self, state):
        root = Node(self.game, self.args, state, visit_count = 1)
        #initialise visit count a 1 car on letend apres
        
        policy, _ = self.model(
            torch.tensor(self.game.get_encoded_state(state), device = self.model.device).unsqueeze(0)
        )
        policy = torch.softmax(policy, axis = 1).squeeze(0).cpu().numpy()
        policy = (1 - self.args['dirichlet_epsilon']) \
            * policy + self.args['dirichlet_epsilon'] \
            * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size)
        # P(s,a) = (1-epsilon) (p\a) + epsilon (eta\a)
        #les lignes precedentes servent a introduire du bruit
        #pour avoir + d'aleatoire et + explorer
        
        valid_moves = self.game.get_valid_moves(state)
        policy *= valid_moves 
        policy /= np.sum(policy)    
        root.expand(policy)
        
        for search in range(self.args['num_searches']):
            #selection
            node = root
            while node.is_fully_expanded():
                node = node.select()
                
                
            #mesure pour la racine     
            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            value = self.game.get_opponent_value(value)
            
            if not is_terminal:
                policy, value = self.model(
                    torch.tensor(self.game.get_encoded_state(node.state),device = self.model.device).unsqueeze(0)
                )
                policy = torch.softmax(policy, axis=1).squeeze(0).cpu().numpy()
                valid_moves = self.game.get_valid_moves(node.state)
                #pour eviter de jouer la ou on a pas le droit
                
                policy *= valid_moves #ca met a 0 les coups illegaux
                policy /= np.sum(policy) # ca remet une proba
                
                value = value.item()
                 
                #expansion
                node = node.expand(policy)
                
                '''
                #simulation
                value = node.simulate()
                '''
                
            #retropropagation
            node.backpropagate(value)    
            
        #on veut maintenant extraire les resulatats : le vecteur de proba
        #on renvoie donc le nb de visite effectuées sur un sommet divisé par le nombre total de visites
        action_probs = np.zeros(self.game.action_size)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        action_probs /= np.sum(action_probs)
        return action_probs

In [9]:
class AlphaZero:
    def __init__(self, model, optimizer, game, args):
        self.model = model
        self.optimizer = optimizer
        self.game = game
        self.args = args
        self.mcts = MCTS(game, args, model)
        
    def selfPlay(self):
        memory = []
        player = 1
        state = self.game.get_initial_state()
        
        while True:
            neutral_state = self.game.change_perspective(state, player)
            #car on veut tjrs etre le joueur 1 pour mcts
            action_probs = self.mcts.search(neutral_state)
            
            memory.append((neutral_state, action_probs, player))
            
            temperature_action_probs = action_probs ** (1 / self.args['temperature'])
            #la temperature permet de modifier durant le processus les coeff dexploration-optimisation
            #si la temperature est est superieure a 1, on explore plus : car la puissance va exacerber ou au contraire reduire les differences
            #plus la temperature est basse, on optimise
            temperature_action_probs = temperature_action_probs / np.sum(temperature_action_probs)
            #car il faut renormaliser derriere
            
            action = np.random.choice(self.game.action_size, p = temperature_action_probs)
            #choisi une action au hasard selon la distribution de proba
            
            state = self.game.get_next_state(state, action, player)
            
            value, is_terminal = self.game.get_value_and_terminated(state, action)
            
            if is_terminal:
                returnMemory = []
                for hist_neutral_state, hist_action_probs, hist_player in memory:
                    hist_outcome = value if hist_player == player else self.game.get_opponent_value(value)
                    returnMemory.append((
                        self.game.get_encoded_state(hist_neutral_state), #encode letat
                        hist_action_probs, #change pas
                        hist_outcome #indique le gagnant du pov du joueur courant
                    ))
                return returnMemory
            
            player = self.game.get_opponent(player)
                
            
    
    def train(self, memory):
        random.shuffle(memory)
        for batchIdx in range(0, len(memory), self.args['batch_size']):
            sample = memory[batchIdx:min(len(memory) - 1, batchIdx + self.args['batch_size'])]
            # Change to memory[batchIdx:batchIdx+self.args['batch_size']] in case of an error
            # sample correspond a tous les elemnts de memory dindices
            # entre batchIdx et batchIdx + batch size
            
            state, policy_targets, value_targets = zip(*sample)
            # names = ['Alice', 'Bob', 'Charlie']
            # ages = [25, 30, 35]
            # zipped = zip(names, ages)
            # print(list(zipped))
            # [('Alice', 25), ('Bob', 30), ('Charlie', 35)]


            
            state, policy_targets, value_targets = np.array(state), np.array(policy_targets), np.array(value_targets).reshape(-1, 1)
            
            state = torch.tensor(state, dtype=torch.float32, device = self.model.device)
            policy_targets = torch.tensor(policy_targets, dtype=torch.float32, device = self.model.device)
            value_targets = torch.tensor(value_targets, dtype=torch.float32, device = self.model.device)
            
            out_policy, out_value = self.model(state)
            
            policy_loss = F.cross_entropy(out_policy, policy_targets)
            value_loss = F.mse_loss(out_value, value_targets)
            loss = policy_loss + value_loss
            
            self.optimizer.zero_grad() # change to self.optimizer
            loss.backward()
            self.optimizer.step() # change to self.optimizer
            
    
    def learn(self):
        for iteration in range(self.args['num_iterations']):
            
            memory = []
            # on stock ici etat, proba, et recompense
            
            self.model.eval()
            # desactive les drop out, ie la mise a zero aleatoires de neurons evitant le surapprentissage
            # contraire de model.train()
            
            # During training, batch norm layers compute the mean 
            # and variance of the current batch to normalize the input.

            # During evaluation, batch norm uses running estimates (computed during 
            # training) instead of the current batch statistics.
            
            for selfPlay_iteration in trange(self.args['num_selfPlay_iterations']):
                memory += self.selfPlay()
                # on stock les donnees de la partie
            # trange donne une visualisation de la progression dans la boucle
            
            self.model.train()
            for epoch in trange(self.args['num_epochs']):
                self.train(memory)
            
            torch.save(self.model.state_dict(), f"model_{iteration}_{self.args['num_searches']}_{self.args['num_iterations']}_{self.args['num_selfPlay_iterations']}_{self.args['num_epochs']}_{self.args['nom']}.pt")
            torch.save(self.optimizer.state_dict(), f"optimizer_{iteration}_{self.args['num_searches']}_{self.args['num_iterations']}_{self.args['num_selfPlay_iterations']}_{self.args['num_epochs']}_{self.args['nom']}.pt")
            # saves the states to a disk file
            # torch.save(obj, f, pickle_module=pickle, pickle_protocol=2, _use_new_zipfile_serialization=True)[source]

In [23]:
game = NDGame(3,2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#implementation ratée : incompatible avece cuda qui est utilisé par pytorch
#GPU Intel intégré (driver i915) Intel UHD Graphics
#GPU vs CPU
# CPU : 10 coeurs chacun faisants differentes operations en //
# GPU : 5000 threads chacune faisant a + b mais avec differents a,b
#cuda : plateforme de calcul en parallele de NVIDIA
 
model = ResNet(game, 4, 64, device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

args = {
    'C': 2,
    'num_searches': 200,
    'num_iterations': 4,
    'num_selfPlay_iterations': 1000,
    'num_epochs': 10,
    'batch_size': 64,
    'temperature': 1.25,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3,
    'nom' : "3²"
}

alphaZero = AlphaZero(model, optimizer, game, args)
alphaZero.learn()

100%|███████████████████████████████████████████| 10/10 [00:20<00:00,  2.03s/it]


In [ ]:
game = NDGame(3,2)
player = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet(game, 4, 64, device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

args = {
    'C': 2,
    'num_searches': 1000,
    'dirichlet_epsilon': 0.,
    'dirichlet_alpha': 0.3
}

model.eval()

mcts = MCTS(game, args, model)

state = game.get_initial_state()


while True:
    print(state)
    
    if player == 1:
        valid_moves = game.get_valid_moves(state)
        print("valid_moves", [i for i in range(game.action_size) if valid_moves[i] == 1])
        action = int(input(f"{player}:"))

        if valid_moves[action] == 0:
            print("action not valid")
            continue
            
    else:
        neutral_state = game.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        action = np.argmax(mcts_probs)
        
    state = game.get_next_state(state, action, player)
    
    value, is_terminal = game.get_value_and_terminated(state, action)
    if is_terminal:
        print(state)
        if value == 1:
            print(player, "won")
        else:
            print("draw")
        break
        
    player = game.get_opponent(player)

In [32]:
#je vais ici simuler 100 parties afin de voir a quelle frequence il y a 
#victoire pour tenter de determiner lexistence de strategies gagnantes

game = NDGame(3,2)


synthese = []

for k in range(4):


    player = 1

    model = ResNet(game, 4, 64, device)
    model.load_state_dict(torch.load( f'model_{k}_200_4_1000_10_3².pt', map_location=device))
    model.eval()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

    args = {
        'C': 2,
        'num_searches': 500,
        'dirichlet_epsilon': 0.,
        'dirichlet_alpha': 0.3
    }
    model.eval()

    mcts = MCTS(game, args, model)




    def joue_contre_soi(n_parties):
        for i in range(n_parties):
            state = game.get_initial_state()
            player = 1
            while True:

                neutral_state = game.change_perspective(state, player)
                mcts_probs = mcts.search(neutral_state)
                action = np.argmax(mcts_probs)
                #print(state)
                #plt.bar(range(game.action_size), mcts_probs)
                #plt.show()

                state = game.get_next_state(state, action, player)

                value, is_terminal = game.get_value_and_terminated(state, action)
                if is_terminal:
                    if value == 1:
                        print(player, "won")
                        resultats.append(player)
                        print(state)
                    else:
                        print("draw")
                        print(state)
                        resultats.append(0)
                    break

                player = game.get_opponent(player)


    def joue_contre_rand(n_parties):
        for i in range(n_parties):
            state = game.get_initial_state()
            player = 1
            while True:
                if player == 1:
                    valid_moves = game.get_valid_moves(state)
                    tab = nd.get_valid_moves(state)
                    indices = np.where(tab == 1)[0]        # prend les indices valant 1
                    action = np.random.choice(indices)

                else:
                    neutral_state = game.change_perspective(state, player)
                    mcts_probs = mcts.search(neutral_state)
                    action = np.argmax(mcts_probs)

                state = game.get_next_state(state, action, player)

                value, is_terminal = game.get_value_and_terminated(state, action)
                if is_terminal:
                    if value == 1:
                        resultats.append(player)
                    else:
                        resultats.append(0)
                    break

                player = game.get_opponent(player)

    resultats = []
    joue_contre_rand(1000)
    nb_zero = resultats.count(0)
    nb_un = resultats.count(1)
    nb_moins_un = resultats.count(-1)
    print("dans simulation ", k)
    print("Nombre de 0 :", nb_zero)
    print("Nombre de 1 :", nb_un)
    print("Nombre de -1 :", nb_moins_un)
 

dans simulation  0
Nombre de 0 : 74
Nombre de 1 : 9
Nombre de -1 : 417
dans simulation  1
Nombre de 0 : 132
Nombre de 1 : 1
Nombre de -1 : 367
dans simulation  2
Nombre de 0 : 160
Nombre de 1 : 0
Nombre de -1 : 340
dans simulation  3
Nombre de 0 : 161
Nombre de 1 : 0
Nombre de -1 : 339


In [ ]:
game = NDGame(3,2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

state = game.get_initial_state()
print(state)
state = game.get_next_state(state, 2, 1)
state = game.get_next_state(state, 4, 1)
state = game.get_next_state(state, 6, -1)
state = game.get_next_state(state, 2, -1)


encoded_state = game.get_encoded_state(state)

tensor_state = torch.tensor(encoded_state, device=device).unsqueeze(0)

model = ResNet(game, 4, 64, device = device)
model.load_state_dict(torch.load('model_2.pt', map_location=device))
model.eval()

policy, value = model(tensor_state)
value = value.item()
policy = torch.softmax(policy, axis=1).squeeze(0).detach().cpu().numpy()

print(value)

print(state)
print(tensor_state)

plt.bar(range(game.action_size), policy)
plt.show()